In [1]:
import pandas as pd
from skimage.io import imread
from skimage import io, data
import matplotlib.pyplot as plt
from tqdm import tqdm
import numpy as np
from skimage.transform import resize
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import Linear, ReLU, CrossEntropyLoss, Sequential, Conv2d, MaxPool2d, Module, Softmax, BatchNorm2d, Dropout
from torch.optim import Adam, SGD

In [2]:
trainData = pd.read_csv('problem2-CNN/train.csv')
testData = pd.read_csv('problem2-CNN/test.csv')

In [3]:
trainData

,filename,width,height,label,xmin,ymin,xmax,ymax
0,11893820-3x2-xlarge.jpg,862,575,bad,12,238,92,318
1,11893820-3x2-xlarge.jpg,862,575,bad,46,42,104,97
2,11893820-3x2-xlarge.jpg,862,575,good,103,24,150,79
3,11893820-3x2-xlarge.jpg,862,575,good,243,2,291,51
4,11893820-3x2-xlarge.jpg,862,575,bad,166,36,228,99
...,...,...,...,...,...,...,...,...
3523,kljlhk.jpg,1548,1032,good,1481,239,1546,353
3524,kljlhk.jpg,1548,1032,good,328,273,361,312
3525,kljlhk.jpg,1548,1032,good,283,245,318,289
3526,kljlhk.jpg,1548,1032,good,226,251,257,289


In [4]:
img_num = trainData['filename']
img_num_test = testData['filename']

In [5]:
# image_path = 'problem2-CNN/images/' + str(filename_1)
# img = imread(image_path)
# plt.imshow(img)

In [6]:
train_data = []

for i in tqdm(range(len(img_num))):
    filename = trainData['filename'][i]
    image_path = 'problem2-CNN/images/' + str(filename)
    img = imread(image_path)
    if img.shape[-1] > 3:
        # discard alpha channel
        img = img[:, :, :3]
    xmin = trainData['xmin'][i]
    xmax = trainData['xmax'][i]
    ymin = trainData['ymin'][i]
    ymax = trainData['ymax'][i]
    #   切割與resize
    partial_img_temp = img[ymin:ymax, xmin:xmax, :]
    partial_img = resize(partial_img_temp, (64, 64))
    
    partial_img = np.reshape(partial_img, (1, 3, 64, 64))
    #   貌似normalize過惹 
    train_data.append(partial_img)
#     io.imshow(partial_img)
#     io.show()

100%|██████████| 3528/3528 [01:45<00:00, 33.35it/s]


In [7]:
test_data = []

for i in tqdm(range(len(img_num_test))):
    filename = testData['filename'][i]
    image_path = 'problem2-CNN/images/' + str(filename)
    img = imread(image_path)
    if img.shape[-1] > 3:
        # discard alpha channel
        img = img[:, :, :3]
    xmin = testData['xmin'][i]
    xmax = testData['xmax'][i]
    ymin = testData['ymin'][i]
    ymax = testData['ymax'][i]
    #   切割與resize
    partial_img_temp = img[ymin:ymax, xmin:xmax, :]
    partial_img = resize(partial_img_temp, (64, 64))
    partial_img = np.reshape(partial_img, (1, 3, 64, 64))
    #   貌似normalize過惹 
    test_data.append(partial_img)

100%|██████████| 394/394 [00:10<00:00, 39.21it/s]


In [8]:
#plt.imshow(train_data[0])

In [9]:
print(train_data[0].shape)

(1, 3, 64, 64)


In [10]:
#train_data_temp = resize(train_data[0], (64, 64))

In [11]:
#plt.imshow(train_data_temp)

In [12]:
#np.max(train_data_temp)

In [13]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.

# N, D_in, H, D_out = 64, 1000, 100, 3
#D_in, H, D_out = 4 * 16 * 16, 100, 3


In [50]:
 class Net(nn.Module):
        def __init__(self, D_in = 4 * 16 * 16, H = 100, D_out = 3):
            super(Net, self).__init__()
            self.CNN = nn.Sequential(
                    # Defining a 2D convolution layer
                Conv2d(3, 4, kernel_size=3, stride=1, padding=1),
      #            BatchNorm2d(4),
                ReLU(inplace=True),
                MaxPool2d(kernel_size=2, stride=2),
                    # Defining another 2D convolution layer
                Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
    #             BatchNorm2d(4),
                ReLU(inplace=True),
                MaxPool2d(kernel_size=2, stride=2),

            )
            self.DNN = nn.Sequential(
                nn.Linear(D_in, H),
                nn.ReLU(),
                nn.Linear(H, D_out),
            )
            
        def forward(self, x):
            x = self.CNN(x)
            x = x.view(x.shape[0], -1)
            x = self.DNN(x)
            return x

In [15]:
loss_func = nn.CrossEntropyLoss()
model = Net()
optimizer = Adam(model.parameters(), lr=0.07)

In [16]:
#test

# train_data_temp = np.reshape(train_data_temp, (1, 3, 64, 64))
# testtest = torch.from_numpy(train_data_temp).float()
# #print(testtest)
# #testtest = Variable(testtest)
# #print(testtest)

# y_pred = CNN(testtest)
# y_pred = y_pred.view(train_data_temp.shape[0], -1)
# y_out = DNN(y_pred)
# #loss = loss_func(y_out)



In [17]:
Y_train_data = trainData['label']
Y_train_data_list = []
for i in range(len(Y_train_data)):
    if Y_train_data[i] == "good":
        Y_train_data_list.append(np.array([0]))
    elif Y_train_data[i] == "bad":
        Y_train_data_list.append(np.array([1]))
    else:
        Y_train_data_list.append(np.array([2]))
Y_train = np.vstack(Y_train_data_list)
        

In [18]:
Y_train.shape

(3528, 1)

In [27]:
Y_train = np.reshape(Y_train, (-1))

In [28]:
Y_train.shape

(3528,)

In [19]:
Y_test_data = testData['label']
Y_test_data_list = []
for i in range(len(Y_test_data)):
    if Y_test_data[i] == "good":
        Y_test_data_list.append(np.array([0]))
    elif Y_test_data[i] == "bad":
        Y_test_data_list.append(np.array([1]))
    else:
        Y_test_data_list.append(np.array([2]))
Y_test = np.vstack(Y_test_data_list)

In [20]:
Y_test.shape

(394, 1)

In [21]:
#因為用append所以是很多np.array, 所以用vstack改成單一np.array
train_data = np.vstack(train_data)

In [22]:
print(train_data.shape)

(3528, 3, 64, 64)


In [51]:
def train_epoch(model, train_data, Y_train, batch_size = 64, epoch = 10):
    model.train()
    for i in range(epoch):
        epoch_loss = 0
        batch_num = len(train_data) // batch_size
        for j in range(batch_num):
            begin = j * batch_size
            end = min(begin + batch_size, train_data.shape[0] - 1)
            X = train_data[begin:end, :, :, :]
            Y = Y_train[begin:end]
            m_batch = end - begin
            
            # converting validation images into torch format
            X = torch.from_numpy(X).float()
            Y = torch.from_numpy(Y)
            
            
            
            #X_afterCNN = CNN(X)
            #X_afterCNN = X_afterCNN.view(X.shape[0], -1)
            #X_afterDNN = DNN(X_afterCNN)
            
            X_afterDNN = model(X)
            
            #print(X_afterDNN)
            #print(Y)
            
            loss = loss_func(X_afterDNN, Y)
            epoch_loss += loss.item()
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        
        print(epoch_loss / train_data.shape[0])

In [52]:
train_epoch(model, train_data, Y_train, batch_size = 64, epoch = 10)

0.009091199981787848
0.00909123251261084
0.009091264595723207
0.009091293739150711
0.009091322451762331
0.009091346923794066
0.009091372240562829
0.009091395910094384
0.009091417763441328
0.00909143782594577


In [ ]:
train_data[953].shape

In [ ]:
plt.imshow(train_data[953])

In [ ]:
np.random.permutation(10)

In [ ]:
a = [1, 2, 3, 4]
np.random.shuffle(a)
a